In [1]:
import os
import datetime
import json
import logging
import warnings
import time
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email import encoders
import smtplib
from io import BytesIO
import pandas as pd

from reportlab.pdfgen import canvas
from reportlab.lib.units import mm
import barcode
from barcode.writer import ImageWriter
from reportlab.lib.utils import ImageReader

import requests
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta

# import azure.functions as func
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

# app = func.FunctionApp()

key_vault_url = "https://premo-vault.vault.azure.net/"
credential = DefaultAzureCredential()
client = SecretClient(vault_url=key_vault_url, credential=credential)

In [3]:
DI_USERNAME = client.get_secret('DI-USERNAME').value
DI_PASSWORD = client.get_secret('DI-PASSWORD').value
SENDER_EMAIL = client.get_secret("GMAIL-USERNAME").value
SENDER_PASSWORD = client.get_secret("GMAIL-APP-PASS").value
BRANCH_EMAILS = json.loads(client.get_secret('BRANCH-EMAILS').value)
_TO_EMAILS_DEV = os.getenv('_TO_EMAILS_DEV', '')
_TO_EMAILS_PROD = os.getenv('_TO_EMAILS_PROD', '')

In [4]:
def create_di_session(DI_USERNAME, DI_PASSWORD):
    session = requests.Session()
    login_url = "https://app.di.no/app/api/login"
    payload = {
        'username': DI_USERNAME,
        'password': DI_PASSWORD
    }

    response = session.post(login_url, data=payload)

    if response.status_code == 200 and "topnavigation" in response.text:
        logging.info("Login successful")
        return session
    else:
        raise Exception("Login failed")

In [5]:
def fetch_di_report(session, report_url, **kwargs):
    """
    Fetches a DI report from the given URL using the provided session and returns it as a pandas DataFrame.

    Parameters:
        session (requests.Session): The session object used to make the HTTP GET request.
        report_url (str): The URL of the report to fetch.
        **kwargs: Optional keyword arguments to pass to pd.read_excel, such as dtype.

    Returns:
        pd.DataFrame: The report data as a pandas DataFrame.

    Raises:
        Exception: If the HTTP request fails (status code not 200).
    """
    try:
        report_response = session.get(report_url)
        if report_response.status_code == 200:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                df = pd.read_excel(
                    BytesIO(report_response.content),
                    engine='openpyxl',
                    **kwargs  # Pass any additional keyword arguments here
                )
            logging.info(f"Report fetched and parsed successfully from {report_url}")
            return df
        else:
            raise Exception(f"Failed to download the file. Status code: {report_response.status_code} for URL: {report_url}")
    except Exception as e:
        logging.error(f"An error occurred while fetching the report: {e}")
        raise

In [6]:
def send_email_with_pdf_from_df(
    sender_email: str,
    sender_password: str,
    to_emails: str,
    cc_emails: str,
    subject: str,
    body: str,
    pdf_bytes: BytesIO = None,
    pdf_filename: str = None,
    max_retries: int = 3
):
    """
    Sends an email with an optional PDF attachment.
    Args:
        sender_email (str): Sender's email address.
        sender_password (str): Sender's email password.
        to_emails (str): Comma-separated recipient email addresses.
        cc_emails (str): Comma-separated CC email addresses.
        subject (str): Subject of the email.
        body (str): Body of the email in HTML format.
        pdf_bytes (BytesIO, optional): In-memory bytes of the PDF file. Defaults to None.
        pdf_filename (str, optional): Filename for the PDF attachment. Defaults to None.
        max_retries (int, optional): Maximum number of retry attempts. Defaults to 3.
    """
    smtp_server = 'smtp.gmail.com'
    smtp_port = 587

    # Create the email message
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = to_emails
    message['Cc'] = cc_emails
    message['Subject'] = subject

    # Attach the email body
    message.attach(MIMEText(body, 'html'))

    # Attach the PDF file if provided
    if pdf_bytes:
        try:
            pdf_bytes.seek(0)
            attachment = MIMEApplication(pdf_bytes.read(), _subtype='pdf')
            attachment_filename = pdf_filename or 'attachment.pdf'  # Determine the filename
            attachment.add_header(
                'Content-Disposition',
                'attachment',
                filename=attachment_filename
            )
            message.attach(attachment)
            logging.info(f"Attached PDF '{pdf_filename}' successfully.")
        except Exception as e:
            logging.error(f"Failed to attach PDF '{pdf_filename}': {e}")
            message.attach(MIMEText('<br><strong>** FAILED TO ATTACH PDF **</strong>', 'html'))

    # Combine recipients
    recipients = to_emails.split(',') + cc_emails.split(',')

    # Attempt to send the email with retries
    for attempt in range(1, max_retries + 1):
        try:
            with smtplib.SMTP(smtp_server, smtp_port) as server:
                server.starttls()
                server.login(sender_email, sender_password)
                server.send_message(message, from_addr=sender_email, to_addrs=recipients)
            logging.info(f"Email sent to {', '.join(recipients)} at {datetime.datetime.now(pytz.timezone('Europe/Stockholm')).strftime('%Y-%m-%d %H:%M')}")
            break
        except Exception as e:
            logging.error(f'Attempt {attempt} failed: {e}')
            if attempt < max_retries:
                logging.info('Retrying in 5 seconds...')
                time.sleep(5)
            else:
                logging.error(f'Failed to send email after {max_retries} attempts')

In [7]:
report_date = datetime.datetime.now(pytz.timezone('Europe/Stockholm')).date() + relativedelta(days=1)
report_date_di_format = datetime.datetime.strftime(report_date, '%d.%m.%Y')
report_url = (f"https://app.di.no/app/ReportGenerator.do?action=getReport&"
            f"selectedReport=291&parameterValue=&index=&token=1724976235436&"
            f"parameterSelectedValues=519&parameterMultiSelectedValues=&"
            f"parameterSelectedValues=7540&parameterMultiSelectedValues=&"
            f"parameterSelectedValues={report_date_di_format}&parameterMultiSelectedValues=&"
            f"parameterSelectedValues=false&parameterMultiSelectedValues=&"
            f"reportFormatAsString=XLSX&reportEncodingAsString=ISO_8859_1")
report_dtype = {'KundeId': str}

In [8]:
session = create_di_session(DI_USERNAME, DI_PASSWORD)
raw_parcel_df = fetch_di_report(session, report_url, dtype=report_dtype)

In [9]:
return_parcel_df = raw_parcel_df[raw_parcel_df['Returkode'].notna()]
return_parcel_sorted_df = return_parcel_df.sort_values(by=['Distrikt'])
return_parcel_sorted_df.head()

,Produkt,Dager,Bedrift,Region,Omraade,Start,Stopp,Distrikt,KundeId,Navn,...,Hoyde,Vekt,FÃ¸rste visning,OkSkannDato,OkSkannStasjon,Aktiv,KundeSysNavn,EksternengangsleveranseId,Sporingsnummer,Returkode
57,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-21,NaN,8017203,00873402216061371032,Annette Bengtsson,...,0,500.0,JA,NaN,NaN,1,Vinted,162742704.0,873402216061371032,E-597103
71,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-21,NaN,8017204,00773402216054137440,Nora Röhr Uhlin,...,0,500.0,JA,NaN,NaN,1,Vinted,162463755.0,773402216054137440,E-589391
121,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-21,NaN,8017207,00373402216070274251,Jacob Brunner,...,0,500.0,JA,NaN,NaN,1,Vinted,162673153.0,373402216070274251,E-595763
122,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-21,NaN,8017207,00473402216000692893,Elsa Bergman,...,0,500.0,JA,NaN,NaN,1,Vinted,162438878.0,473402216000692893,E-587477
123,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-21,NaN,8017207,00473402216033028058,Elsa Bergman,...,0,500.0,JA,NaN,NaN,1,Vinted,162355604.0,473402216033028058,E-584208


In [11]:
## DEBUG, DON't USE IN PRODUCTION ##
debug_return_parcel_sorted_df = return_parcel_sorted_df[return_parcel_sorted_df['Omraade'].isin(['ESSINGEN', 'MATTEUS', 'JAKOB'])]
debug_return_parcel_sorted_df

,Produkt,Dager,Bedrift,Region,Omraade,Start,Stopp,Distrikt,KundeId,Navn,...,Hoyde,Vekt,FÃ¸rste visning,OkSkannDato,OkSkannStasjon,Aktiv,KundeSysNavn,EksternengangsleveranseId,Sporingsnummer,Returkode
57,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-21,NaN,8017203,00873402216061371032,Annette Bengtsson,...,0,500.0,JA,NaN,NaN,1,Vinted,162742704.0,873402216061371032,E-597103
71,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-21,NaN,8017204,00773402216054137440,Nora Röhr Uhlin,...,0,500.0,JA,NaN,NaN,1,Vinted,162463755.0,773402216054137440,E-589391
121,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-21,NaN,8017207,00373402216070274251,Jacob Brunner,...,0,500.0,JA,NaN,NaN,1,Vinted,162673153.0,373402216070274251,E-595763
122,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-21,NaN,8017207,00473402216000692893,Elsa Bergman,...,0,500.0,JA,NaN,NaN,1,Vinted,162438878.0,473402216000692893,E-587477
123,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-21,NaN,8017207,00473402216033028058,Elsa Bergman,...,0,500.0,JA,NaN,NaN,1,Vinted,162355604.0,473402216033028058,E-584208
124,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-21,NaN,8017207,00473402216037673179,Elsa Bergman,...,0,2000.0,JA,NaN,NaN,1,Vinted,162608212.0,473402216037673179,E-592864
125,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-21,NaN,8017207,00573402216078164992,Elsa Bergman,...,0,2000.0,JA,NaN,NaN,1,Vinted,162755164.0,573402216078164992,E-597437
126,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-21,NaN,8017207,00673402216063445195,Agnes Adin,...,0,1000.0,JA,NaN,NaN,1,Vinted,162828892.0,673402216063445195,E-599463
127,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-21,NaN,8017207,00873402216029845346,Elsa Bergman,...,0,500.0,JA,NaN,NaN,1,Vinted,162337485.0,873402216029845346,E-583064
128,RETUR,NaN,Premo,Sthlm City,ESSINGEN,2024-11-21,NaN,8017207,00973402216007687965,Alice Jakobsson Åkesson,...,0,2000.0,JA,NaN,NaN,1,Vinted,162812583.0,973402216007687965,E-598935


In [12]:
def create_label(c, row):
    barcode_value = str(row['Returkode'])
    code128 = barcode.get('code128', barcode_value, writer=ImageWriter())
    barcode_image = code128.render(writer_options={'write_text': False})

    # Create an ImageReader from the PIL Image
    image_reader = ImageReader(barcode_image)
   
    # Draw the label content on the canvas
    c.setFont("Helvetica", 12)
    c.drawString(10*mm, 80*mm, f"Region: {row['Region']}")
    c.drawString(10*mm, 70*mm, f"Omraade: {row['Omraade']}")
    c.drawString(10*mm, 60*mm, f"Distrikt: {row['Distrikt']}")
   
    c.setFont("Helvetica-Bold", 24)
    returkode_text = str(row['Returkode'])
    text_width = c.stringWidth(returkode_text, "Helvetica-Bold", 24)
    x_position = (192*mm - text_width) / 2
    c.drawString(x_position, 40*mm, returkode_text)
   
    c.drawImage(image_reader, 50*mm, 10*mm, width=92*mm, height=20*mm)

def generate_labels_pdf(df):
    pdf_buffer = BytesIO()
    c = canvas.Canvas(pdf_buffer, pagesize=(192*mm, 102*mm))

    for index, row in df.iterrows():
        create_label(c, row)
        c.showPage()

    c.save()
    pdf_buffer.seek(0)
    return pdf_buffer

In [13]:
pdf_label_buffer = generate_labels_pdf(debug_return_parcel_sorted_df)

In [14]:
send_email_with_pdf_from_df(
    sender_email=SENDER_EMAIL,
    sender_password=SENDER_PASSWORD,
    to_emails='johan.tokarskij@premo.se',
    cc_emails='',
    subject='Barcodes för upphämtade paket',
    body='',
    pdf_bytes=pdf_label_buffer,
    pdf_filename='labels.pdf'
)